In [1]:
import gensim

In [3]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.chdir("/content/drive/MyDrive")
xml_data = open('./dataset/articles-training-byarticle-20181122.xml', 'r').read()  # Read file
root = ET.XML(xml_data)

data = []

for i, child in enumerate(root):
    data.append([subchild.text for subchild in child])

df = pd.DataFrame(data).T  # Write in DF and transpose it

print(data[218])

['Hollywood will not easily recover from Harvey Weinstein -- not for a long time. The hypocrisy level has hit Defcon 1, 9.9 on the Richter scale.', "Hollywood’s politics have always been a self-serving charade, a liberal masquerade for a rapacious and lubricious lifestyle. But now, thanks to the Weinstein scandal, we see it more clearly than ever. And it couldn't be more repellent. (I had always thought Bill Clinton would have made the greatest studio executive of all time. Now I'm convinced of it.)", 'If conservative investors had any courage, this would be the time to make a hostile takeover of the movie business. Unfortunately, they don’t. I know this from bitter personal experience. Wealthy conservatives are delighted to support the Philharmonic, but when it comes to popular culture they turn away, as if afraid to get their hands dirty.', 'That this is a huge mistake should be obvious. They have abandoned the culture -- and our children -- to the creepiest people imaginable. What i

In [5]:
xml_data1 = open('./dataset/ground-truth-training-byarticle-20181122.xml', 'r').read()  # Read file
root1 = ET.XML(xml_data1)

labels = []

for i, child in enumerate(root1):
    if child.attrib['hyperpartisan']=='true':
        labels.append(1)
    else:
        labels.append(0)

print(labels)

[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [26]:
training_data = data[0:500]
training_labels = labels[0:500]

testing_data = data[500:640]
testing_labels = labels[500:640]

In [27]:
train_X = []
train_Y = []

sum = 0
for i in range(len(training_data)):
  sentences = []
  for j in range(len(training_data[i])):

    if training_data[i][j] is None:
      continue
    if(len(training_data[i][j])<3):
      continue

    # print(data[i][j])
    lst = sent_tokenize(training_data[i][j])

    for k in range(len(lst)):
      sentences.append(lst[k])

  tokenized_context_send = []

  for j in range(len(sentences)):

    raw_context = sentences[j]
    tokenized_context1 = raw_context.split()

    new_list = []
    for word in tokenized_context1:
      word1 = word.lower()

      new_word = ""

      for p in word1:
        if (p.isalpha()):
          new_word+=p

      if(len(new_word)>2 and new_word!="the"):
        new_list.append(new_word)

    if(len(new_list)>1):
      tokenized_context_send.append(new_list)

  # print(tokenized_context_send)
  
  train_X.append(tokenized_context_send)
  train_Y.append(training_labels[i])


In [28]:
test_X = []
test_Y = []

sum = 0
for i in range(len(testing_data)):
  sentences = []
  for j in range(len(testing_data[i])):

    if testing_data[i][j] is None:
      continue
    if(len(testing_data[i][j])<3):
      continue

    # print(data[i][j])
    lst = sent_tokenize(testing_data[i][j])

    for k in range(len(lst)):
      sentences.append(lst[k])

  tokenized_context_send = []

  for j in range(len(sentences)):

    raw_context = sentences[j]
    tokenized_context1 = raw_context.split()

    new_list = []
    for word in tokenized_context1:
      word1 = word.lower()

      new_word = ""

      for p in word1:
        if (p.isalpha()):
          new_word+=p

      if(len(new_word)>2 and new_word!="the"):
        new_list.append(new_word)

    if(len(new_list)>1):
      tokenized_context_send.append(new_list)

  # print(tokenized_context_send)
  
  test_X.append(tokenized_context_send)
  test_Y.append(testing_labels[i])

In [29]:
sentences = []
for i in range(len(train_X)):
  for j in range(len(train_X[i])):
    sentences.append(train_X[i][j])

for i in range(len(test_X)):
  for j in range(len(test_X[i])):
    sentences.append(test_X[i][j])

# print(sentences[0:10])

In [22]:
word_model = gensim.models.Word2Vec(sentences, size=100 , min_count=1 , window=5, iter=100)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
for word in ['model', 'network', 'train', 'learn']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) 
                           for similar, dist in word_model.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))

def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

Result embedding shape: (19291, 100)
Checking similar words:
  model -> aggrandizement (0.48), coal (0.45), earmarked (0.44), patients (0.44), standby (0.44), sensefor (0.44), miner (0.44), careers (0.43)
  network -> spokesman (0.52), antidefamation (0.47), abc (0.46), warner (0.45), bureaus (0.45), rulebook (0.45), pitcavage (0.45), cooper (0.43)
  train -> destroys (0.63), awayto (0.56), districts (0.54), voids (0.54), moorpark (0.54), burbank (0.53), kitchen (0.50), kan (0.49)
  learn -> read (0.43), bothered (0.42), ryanhes (0.41), gripe (0.41), writes (0.41), preach (0.41), dickersons (0.40), toss (0.39)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [ ]:
print(pretrained_weights[0])

In [ ]:
final_train_X = []
final_train_Y = []
for i in range(len(train_X)):
  feature_vector = []
  for j in range(len(train_X[i])):
    
    sm = 0
    length = len(train_X[i][j])
    for words in train_X[i][j]:
      weight = pretrained_weights[word2idx(words)]
    
      sm1 = 0
      for k in range(len(weight)):
        sm1=sm1+weight[k]

      sm = sm + sm1/len(weight)

    feature_vector.append(sm)

  print(feature_vector)
  final_train_X.append(feature_vector)
  final_train_Y.append(train_Y[i])

In [39]:
final_test_X = []
final_test_Y = []
for i in range(len(test_X)):
  feature_vector = []
  for j in range(len(test_X[i])):
    
    sm = 0
    length = len(test_X[i][j])
    for words in test_X[i][j]:
      weight = pretrained_weights[word2idx(words)]
    
      sm1 = 0
      for k in range(len(weight)):
        sm1=sm1+weight[k]

      sm = sm + sm1/len(weight)

    feature_vector.append(sm)

  final_test_X.append(feature_vector)
  final_test_Y.append(test_Y[i])

In [93]:

# print(np.array(final_train_X).shape)
print(len(final_train_X[0]))
print(final_train_Y)

60
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

In [84]:
from keras.preprocessing.sequence import pad_sequences

train_Xp = pad_sequences( final_train_X , maxlen=100 , dtype='float32')
test_Xp = pad_sequences( final_test_X , maxlen=100 , dtype='float32')


In [79]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout , Activation , Bidirectional
from keras.layers import SimpleRNN

In [99]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

for i in range(len(train_Xp)):
  train_Xp[i] = softmax(train_Xp[i])

for i in range(len(test_Xp)):
  test_Xp[i] = softmax(test_Xp[i])

In [ ]:
model = Sequential()


model.add(Embedding(input_dim=vocab_size,
                    output_dim=emdedding_size,
                    weights=[pretrained_weights],
                    input_length=100,
                    mask_zero=True,
                    trainable=False))

model.add(Bidirectional(LSTM(100)))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_X1 = np.asarray(train_Xp)
train_Y1 = np.asarray(final_train_Y)

print(train_X1)
print(train_Y1)

history = model.fit(train_X1, train_Y1, epochs=5, batch_size=64, verbose=1)

In [110]:
test_X1 = np.asarray(test_Xp)
test_Y1 = np.asarray(final_test_Y)


scores = model.evaluate(test_X1, test_Y1, verbose=0)
print(scores[1])

0.7285714149475098


In [113]:
from sklearn.svm import SVC
model1 = SVC(C=0.1 , gamma='scale' , kernel='rbf')

model1.fit(train_X1,train_Y1)
y1_predict = model1.predict(test_X1)


In [115]:

print(classification_report(test_Y1, y1_predict))
print('-----------------------------------------------------------')
print(f"Accuracy: {round(accuracy_score(test_Y1, y1_predict), 2)}") 

              precision    recall  f1-score   support

           0       0.73      1.00      0.84       102
           1       0.00      0.00      0.00        38

    accuracy                           0.73       140
   macro avg       0.36      0.50      0.42       140
weighted avg       0.53      0.73      0.61       140

-----------------------------------------------------------
Accuracy: 0.73


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
